In [ ]:
# --- STEP 1: Install dependencies ---
!pip install yt-dlp pydub tqdm ipywidgets > /dev/null

# --- STEP 2: Import libraries ---
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor
import ipywidgets as widgets
from IPython.display import display, FileLink, clear_output
import shutil

# --- STEP 3: Prepare output folder ---
output_dir = "mp3_downloads"
os.makedirs(output_dir, exist_ok=True)

# --- STEP 4: Define the download function ---
def download_song(term, progress_widgets):
    """
    Download a single song and update progress widgets.
    """
    safe_name = term.replace(" ", "_")[:60]
    expected_file = os.path.join(output_dir, f"{safe_name}.mp3")

    if os.path.exists(expected_file):
        progress_widgets['skipped'].value += 1
        progress_widgets['status'].value = f"⏭️ Skipped: {term}"
        return (term, "skipped")

    cmd = [
        "yt-dlp",
        f"ytsearch1:{term}",
        "-x",
        "--audio-format", "mp3",
        "--audio-quality", "0",
        "-o", f"{output_dir}/%(title)s.%(ext)s"
    ]

    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        progress_widgets['success'].value += 1
        progress_widgets['status'].value = f"✔️ Downloaded: {term}"
        return (term, "success")
    except subprocess.CalledProcessError:
        progress_widgets['failed'].value += 1
        progress_widgets['status'].value = f"❌ Failed: {term}"
        return (term, "failed")

# --- STEP 5: Create UI widgets ---
file_upload = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description="Upload song list"
)

download_button = widgets.Button(
    description="Start Download",
    button_style="success",
    icon="download"
)

progress_success = widgets.IntProgress(value=0, min=0, max=1, description='Success')
progress_skipped = widgets.IntProgress(value=0, min=0, max=1, description='Skipped')
progress_failed = widgets.IntProgress(value=0, min=0, max=1, description='Failed')
status_label = widgets.Label(value="Status: Waiting for upload")

output_area = widgets.Output()

# --- STEP 6: Define download process ---
def start_download(b):
    clear_output(wait=True)
    display(file_upload, download_button, progress_success, progress_skipped, progress_failed, status_label, output_area)

    if not file_upload.value:
        with output_area:
            print("❌ Please upload a text file first.")
        return

    uploaded_filename = list(file_upload.value.keys())[0]
    content = file_upload.value[uploaded_filename]['content']
    search_terms = [line.decode('utf-8').strip() for line in content.splitlines() if line.strip()]

    total_songs = len(search_terms)
    progress_success.max = total_songs
    progress_skipped.max = total_songs
    progress_failed.max = total_songs

    progress_widgets = {
        'success': progress_success,
        'skipped': progress_skipped,
        'failed': progress_failed,
        'status': status_label
    }

    with output_area:
        print(f"🎶 Found {total_songs} songs in {uploaded_filename}\n")

    # Multi-threaded downloads
    MAX_WORKERS = 4
    results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(download_song, term, progress_widgets) for term in search_terms]
        for future in futures:
            results.append(future.result())

    # Summary
    success = [term for term, status in results if status == "success"]
    skipped = [term for term, status in results if status == "skipped"]
    failed = [term for term, status in results if status == "failed"]

    with output_area:
        print("\n✅ Downloading complete!\n")
        print(f"📊 Summary:")
        print(f"  ✔️ Successful: {len(success)}")
        print(f"  ⏭️ Skipped: {len(skipped)}")
        print(f"  ❌ Failed: {len(failed)}")
        if failed:
            print("\n⚠️ Could not download:")
            for f in failed:
                print(f"   - {f}")

        # Zip the results
        shutil.make_archive("mp3_downloads", 'zip', output_dir)
        print("\n📦 Download your MP3s:")
        display(FileLink("mp3_downloads.zip"))

# --- STEP 7: Wire button to function ---
download_button.on_click(start_download)

# --- STEP 8: Display UI ---
display(file_upload, download_button, progress_success, progress_skipped, progress_failed, status_label, output_area)
